In [ ]:
# The examples in this notebook use a set of nine benchmarks described in our publication.
# These benchmarks can be downloaded via FTP from: ftp.cs.huji.ac.il/users/nadavb/protein_bert/protein_benchmarks
# Download the benchmarks into a directory on your machine and set the following variable to the path of that directory.
BENCHMARKS_DIR = './datasets/'
import os

import pandas as pd
from IPython.display import display

from tensorflow import keras

from sklearn.model_selection import train_test_split

from model_code import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, finetune, evaluate_by_len
from model_code.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs


In [ ]:

BENCHMARK_NAME = 'DBP_Predict_refine_13289_230726'

# A local (non-global) bianry output
OUTPUT_TYPE = OutputType(False, 'binary')
UNIQUE_LABELS = [0, 1]
OUTPUT_SPEC = OutputSpec(OUTPUT_TYPE, UNIQUE_LABELS)


# Loading the dataset

train_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.train.csv' % BENCHMARK_NAME)
train_set = pd.read_csv(train_set_file_path).dropna().drop_duplicates()
train_set, valid_set = train_test_split(train_set, stratify = train_set['label'], test_size = 0.04, random_state = 0)

test_set_file_path = os.path.join(BENCHMARKS_DIR, 'DBP_Predict_PDB.test.csv')
test_set = pd.read_csv(test_set_file_path).dropna().drop_duplicates()

print(f'{len(train_set)} training set records, {len(valid_set)} validation set records, {len(test_set)} test set records.')


# Loading the pre-trained model and fine-tuning it on the loaded dataset

pretrained_model_generator, input_encoder = load_pretrained_model()

# get_model_with_hidden_layers_as_outputs gives the model output access to the hidden layers (on top of the output)
model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
        get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)

training_callbacks = [
    keras.callbacks.ReduceLROnPlateau(patience = 1, factor = 0.25, min_lr = 1e-05, verbose = 1),
    keras.callbacks.EarlyStopping(patience = 2, restore_best_weights = True),
]

finetune(model_generator, input_encoder, OUTPUT_SPEC, train_set['seq'], train_set['label'], valid_set['seq'], valid_set['label'], \
        seq_len = 256, batch_size = 32, max_epochs_per_stage =10, lr = 1e-04, begin_with_frozen_pretrained_layers = True, \
        lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 1024, final_lr = 1e-05, callbacks = training_callbacks)


# Evaluating the performance on the test-set



In [ ]:

X_test,y_preds,y_trues,results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_set['protein_ID'],test_set['seq'], test_set['label'], \
        start_seq_len = 512, start_batch_size = 32)

print('Test-set performance:')
display(results)

print('Confusion matrix:')
display(confusion_matrix)

dataframe_512 = pd.DataFrame()
dataframe_1024 = pd.DataFrame()
dataframe_2048 = pd.DataFrame()


dataframe_512['protein_ID'] = X_test[0]
dataframe_512['y_preds'] = y_preds[0]
dataframe_512['y_trues'] = y_trues[0]
dataframe_1024['protein_ID'] = X_test[1]
dataframe_1024['y_preds'] = y_preds[1]
dataframe_1024['y_trues'] = y_trues[1]
dataframe_2048['protein_ID'] = X_test[2]
dataframe_2048['y_preds'] = y_preds[2]
dataframe_2048['y_trues'] = y_trues[2]

dataframe = pd.concat([dataframe_512,dataframe_1024,dataframe_2048])

dataframe.to_csv('./PLM-DBPPred_predict_result.csv')

In [ ]:
from scipy.stats import spearmanr
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix,precision_score,recall_score,f1_score
import numpy as np

y_pred = np.array(dataframe['y_preds'])
# print(y_pred)

y_true = dataframe['y_trues'].values

y_pred_classes = (y_pred >= 0.5)
y_true_classes = (y_true >= 0.5)

# print(y_pred_classes)
# print(y_true_classes)
results = pd.DataFrame()
results['AUC'] = [roc_auc_score(y_true, y_pred)]
results['Accuracy'] = [accuracy_score(y_true, y_pred_classes)]
results['precision'] = [precision_score(y_true, y_pred_classes)]
results['recall_score'] = [recall_score(y_true, y_pred_classes)]
results['f1_score'] = [f1_score(y_true, y_pred_classes)]

tn, fp, fn, tp = confusion_matrix(y_true,y_pred_classes).ravel()

results['sensitivity'] = [tp / (tp + fn)]
results['specificity'] = [tn / (tn + fp)]

confusion_matrix = pd.DataFrame(confusion_matrix(y_true,y_pred_classes))
confusion_matrix
results


In [ ]:
import pickle
with open('./DBP_model_param/model.pt', 'wb') as f:
    pickle.dump(model_generator, f)

with open('./DBP_model_param/model_OUTPUT_SPEC.pt', 'wb') as f:
    pickle.dump(OUTPUT_SPEC, f)

with open('./DBP_model_param/model_input_encoder.pt', 'wb') as f:
    pickle.dump(input_encoder, f)